In [1]:
import numpy as np
import cv2
from numpy.lib import stride_tricks
import matplotlib.pyplot as plt

In [2]:
### here 16 stands for bits
## say "dtype= ___number", then nuber of bytes each item will take is number/8
## ie each item will occupy memory of 16/8=2 bytes.!! 
## eg dtype=float64, then item will take 8-bytes of memory.!

############ NOTE BYTES WILL BE SAME IRRESPECTIVE IF ITS UINT OR INT OR FLOAT ###########

# X = np.array([[1,2,3,4,5],[11,22,33,44,55],[111,222,333,444,555]], dtype='float16')
X = np.array([[0,1,2],[3,4,5]], dtype='int16')
print("bytesize of each element ",X.itemsize)

bytesize of each element  2


In [3]:
# Numpy arrays are stored in a single contiguous (continuous) block of memory. 
# There are two key concepts relating to memory: dimensions and strides.
# Strides are the number of bytes you need to step in each dimension when traversing the array.
print(X.strides)

#### AS U GO FURTHER U WILL KNOW WHAT IT SIGNIFYS.!!!

(6, 2)


In [ ]:
### now (6,2) actually tells u that :- 
## u need to skip 6 bytes to acess this array [3,4,5]
## u need to skip 2 bytes to access the elements in [0,1,2]


In [ ]:
s=np.random.randint(0,12,size=(8,19),dtype = np.uint8)
print(s.strides)
# if u want to access those 19-sized vectors u need to have stride 19
# if u want to access an element in those 19-sized vector, the u have stride 1
ss=np.random.randint(0,12,size=(8,19),dtype = np.uint64)
print(ss.strides)
# if u want to access those 19-sized vectors u need to have stride 19*8(each element occupys 8 bytes)
# if u want to access an element in those 19-sized vector, the u have stride 8

#### NOTE   # The stride in the last dimension is always equal to the itemsize.

################# AS U CHANGE THE BYTE-SIZE OF AN ELEMENT, YOUR STRIDES GET MULTIPLYED BY THAT BYTE.

## eg- For s, byte was 1 since bits were 8 .... For ss, byte is 8 since bits are 64..!
## and respectively , strides got multiplyd by that BYTE since u change the BITS value.!! 



In [ ]:
qw=np.random.randint(0,25,size=(4,2,5),dtype=np.int8)

## strides will be 10,5,1 here. if u have not understood how, then go back.!
## there's a simple trick to actually find the strides without going into theory, 
## but its good to know the theory first and then use the trick.!!

### if u want to know that trick (OR PATTERN), just keep on increasing dimensions 
## and print their strides, the pattern will be visible.!!!
print(qw.strides)

In [ ]:
big_arr=np.random.randint(0,25,size=(3,4,5,72),dtype=np.int64)
## guess the strides ehre.!
## 72*5*4, 72*5, 72, 1 if the bits are 8.
## if bits are 64, then [ 72*5*4*8, 72*5*8, 72*8, 1*8 ] will be the strides.
### VERIFYING ABOVE STATEMENT
print("Calculated",72*5*4*8,72*5*8,72*8,1*8,)
print("numpy function",big_arr.strides)

In [4]:
### NOW COMES NUMPY.STRIDE_TRICKs. BE CAREFUL
## TRY TO UNDERTAND MOST OF THIS, BCUZ ONCE U KNOW THIS BASICS, U CAN CODE CONV,MAXPOOLING, AND WHAT NOT ?? ####

In [ ]:
### say u want ato perform a 10x10 convolution on an image without FOR loops
size=10
img=cv2.imread('dog.14.jpg',0)
#img=cv2.resize(img,(500,1000))
shape = (img.shape[0] - size + 1, img.shape[1] - size + 1, size, size)
print('orig img shape',img.shape)
print('Shape u want',shape)
strides=img.strides*2
print('The strides u lookig for',strides)

patches = stride_tricks.as_strided(img, shape=shape, strides=strides)
res_img =np.mean(patches,axis=(-1,-2))
print(res_img.shape,res_img.dtype)
cv2.imshow('Convolved image',res_img.astype(np.uint8))

cv2.imshow('original_img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
###### PRACTISIZNG ON 1-D ARRAYS FIRST AND THEN MOVING TO 2-D ARRAYS

##### NOW THIS IS JUST 1-D CONVOLUTION WHICH INCLUDES CHNGINF THE WINDOW SIZE AS WELL AS THE STRIDE LENGTH

x=np.sin(np.arange(0,50,dtype=np.uint8))
size=x.itemsize
window=3
stride=1
new_len=1+(len(x)-window)//stride
print("Shape u get based on stride and window ",new_len,window)
print(x.shape)
new_x = stride_tricks.as_strided(x,shape=(new_len,window),strides=(stride*size,1*size))
#print(new_x)
## if u want to really implement conv whose window is [1,1,1]/3 then just do, np.mean(new_x,axis=1)
conv=np.mean(new_x,axis=1)
plt.plot(x)
plt.plot(conv)

In [5]:
##### THIS IS A SIMPLE MAXPOOLING EXAMPLE WITH 4X8 ARRAY.
img=np.asarray([[1,2,3,4,5,6,7,8],
                [11,22,33,44,55,66,77,88],
                [111,222,333,444,555,666,777,888],
                [-1,2,-3,-4,-5,-6,-7,-8]])#
bite=img.itemsize
print("img shape is ", img.shape)
stride=2
shape = (img.shape[0]//2, img.shape[1]//2, stride, stride)
print('based on stride, u will get the shape',shape)
print('Strides in img',img.strides)

############ TRY CHANGINF THOSE NUMBERS IF U DONT UNDERSTAND.
#### TWEAK THEM IN  A WAY THAT U WILL UNDERSTAND TO THE FULLEST........!!!!!!!!!!!
max_pool=stride_tricks.as_strided(img,shape=shape,strides=(img.shape[-1]*2*bite,stride*bite,img.shape[-1]*bite,1*bite))

print(max_pool[1,3])
print(img[2:4,6:8])

img shape is  (4, 8)
based on stride, u will get the shape (2, 4, 2, 2)
Strides in img (32, 4)
[[777 888]
 [ -7  -8]]
[[777 888]
 [ -7  -8]]


In [ ]:
### THIS IS MAX-POOLING ON ANY IMAGE BUT GRAYSCALE
img=cv2.imread('dog.14.jpg',0)
h,w=img.shape[:2]
bite=img.itemsize
print("img shape is ", h,w)
stride=2
shape = (img.shape[0]//2, img.shape[1]//2, stride, stride)
print('based on stride, u will get the shape',shape)
print('Strides in img',img.strides)
max_pool=stride_tricks.as_strided(img,shape=shape,strides=(w*2*bite,stride*bite,w*bite,1*bite))
cv2.imshow('re',np.max(max_pool,axis=(-1,-2)).astype(np.uint8))#
cv2.waitKey(0)
cv2.destroyAllWindows()